In [1]:
import torch
import torch.nn as nn
# from numpy import exp, sqrt
from numpy.random import normal
from torch import exp, sqrt, randn_like
from rdkit import Chem

In [5]:
LATENT_DIM = 4      # arbitrary
MAX_SIZE = 22*3     # try first with (x,y,z) for each atom

class MoleculeVAE(nn.Module):
    def __init__(self): # maybe define encoder, decoder, in_channels, out_channels in constructor too
        # self.decoder = DefaultDecoder() if decoder is None else decoder
        # maybe pass in data as reaction frame here too?
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(MAX_SIZE, 32),
            nn.ReLU(),
            nn.Linear(32, 2*LATENT_DIM)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2*LATENT_DIM, 32),
            nn.ReLU(),
            nn.Linear(32, MAX_SIZE)
        )
    
    def reparameterise(self, mean_z, log_var_z):
        if self.training:
            eps = randn_like() # is this meant to be randn_like(log_var_z)?
            z = mean_z + eps * sqrt(exp(log_var_z))
            return z
        else:
            return mean_z
    
    def forward(self, x): # note: might be easier to do this the other way explicitly defining mean and var
        # reshape input into a vector, then reshape using view(-1, batchsize=2, d)
        params_z = self.encoder(x.view(-1, input_size)).view(-1, 2, LATENT_DIM) # this encoder may need to be changed
        mean_z = params_z[:, 0, :]
        log_var_z = params_z[:, 1, :]
        z = self.reparameterise(mean_z, log_var_z)
        return self.decoder(z), mean_z, log_var_z 

    # x_hat = model TS; x = TS
    def loss_func(generated_TS, real_TS, mean_z, log_var_z, beta=1):
        BCE = nn.functional.binary_cross_entropy(generated_TS, real_TS.view(-1, MAX_SIZE), reduction='sum') 
        KLD = 0.5 * torch.sum(exp(log_var_z) - log_var_z - 1 + mean_z**2)
        return BCE + beta * KLD

# define device and model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MoleculeVAE().to(device)

In [6]:
# in cell below, need to figure out how to load sdf data into loader
# look at pytorch data handling https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

# load train and test data and convert to RDKit mols in a list
base_folder = 'data/'
train_r_file = base_folder + 'train_reactants.sdf'
train_ts_file = base_folder + 'train_ts.sdf'
train_p_file = base_folder + 'train_products.sdf'
test_r_file = base_folder + 'test_reactants.sdf'
test_ts_file = base_folder + 'test_ts.sdf'
test_p_file = base_folder + 'test_products.sdf'
# train
train_r = Chem.ForwardSDMolSupplier(train_r_file, removeHs=False, sanitize=False)
train_r = [x for x in train_r]
train_ts = Chem.ForwardSDMolSupplier(train_ts_file, removeHs=False, sanitize=False)
train_ts = [x for x in train_ts]
train_p = Chem.ForwardSDMolSupplier(train_p_file, removeHs=False, sanitize=False)
train_p = [x for x in train_p]
# test
test_r = Chem.ForwardSDMolSupplier(test_r_file, removeHs=False, sanitize=False)
test_r = [x for x in test_r]
test_ts = Chem.ForwardSDMolSupplier(test_ts_file, removeHs=False, sanitize=False)
test_ts = [x for x in test_ts]
test_p = Chem.ForwardSDMolSupplier(test_p_file, removeHs=False, sanitize=False)
test_p = [x for x in test_p]

In [20]:
class ReactionDatasetInstance(torch.utils.data.Dataset):
    """Creates instance of reaction dataset. Has functions for train and test sets."""
    
    def __init__(self, train_r_name, train_ts_name, train_p_name, 
                       test_r_name,  test_ts_name,  test_p_name, base_folder='data/'):
        self.base_folder = base_folder
        train_r_file = base_folder + train_r_name
        train_ts_file = base_folder + train_ts_name
        train_p_file = base_folder + train_p_name
        test_r_file = base_folder + test_r_name
        test_ts_file = base_folder + test_ts_name
        test_p_file = base_folder + test_p_name
        self.train_r, self.train_ts, self.train_p = create_training_set(train_r_file, train_ts_file, train_p_file)
        self.test_r, self.test_ts, self.test_p = create_test_set(test_r_file, test_ts_file, test_p_file)

    def create_training_set(self, train_r_file, train_ts_file, train_p_file):
        train_r = sdf_to_rdmol(train_r_file)
        train_ts = sdf_to_rdmol(train_ts_file)
        train_p = sdf_to_rdmol(train_p_file)
        return train_r, train_ts, train_p
    
    def create_test_set(self, test_r_file, test_ts_file, test_p_file):
        test_r = sdf_to_rdmol(test_r_file)
        test_ts = sdf_to_rdmol(test_ts_file)
        test_p = sdf_to_rdmol(test_p_file)
        return test_r, test_ts, test_p
    
    def sdf_to_rdmol(self, geometry_file):
        geometries = Chem.ForwardSDMolSupplier(geometry_file, removeHs=False, sanitize=False)
        geometries = [x for x in geometries]

    def coordinate_to_interatomic_dist():
        return

    # visualisation funcs here, eda subclass?
    #  compare averages

    # need option to display as whole reaction (r, ts, p) for train and test
        # perhaps can identify scaffold bias at some point on the split
        # can visualise how the interatomic dynamics change
    
    # should have some way of comparing models formally? class, etc.?
        # compare ts_gen TS: average, initial guess, final estimate, real.
            # where is there most room for improvement
    


In [9]:
### put R-TS-P into dataloader

# really, it's train_r, train_p, val_ts then test_r, test_ts, test_p

# zip rs and ps
train_rxn_endpoints = list(zip(train_r, train_p))


# class ReactionDataset()


list

In [13]:
# setting optimiser
learning_rate = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

# training and testing the VAE
epochs = 5
codes = dict(mean=list(), log_var=list(), y=list())
for epoch in range(0, epochs+1):
    # training
    if epoch > 0:
        model.train()
        train_loss = 0
        for real_TS, _ in train_loader:
            real_TS = real_TS.to(device)
            # === forward ===
            generated_TS, mean_z, log_var_z = model(x)
            loss = loss_func(generated_TS, real_TS, mean_z, log_var_z)
            train_loss += loss.item()
            # === backward ===
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        # === log ===
        print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
        
    # testing
    means, log_vars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, y in test_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            test_loss += loss_function(x_hat, x, mean, log_var).item()
            # === log ===
            means.append(mean.detach())
            log_vars.append(log_var.detach())
            labels.append(y.detach())
    # === log ===
    codes['mean'].append(torch.cat(means))
    codes['log_var'].append(torch.cat(log_vars))
    codes['y'].append(torch.cat(labels))
    test_loss /= len(test_loader.dataset)
    print(f'===> Test set loss: {test_loss:.4f}')
    display_images(x, x_hat, 1, f'Epoch {epoch}')

NameError: name 'test_loader' is not defined

In [ ]:
class VAE(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            # linear (size of input, 2d), size of input= max possible size i.e. largest mol
            nn.Linear(input_size, d**2),
            nn.ReLU(),
            nn.Linear(d ** 2, d * 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(d, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, input_size)
            # would use sigmoid here if input was between 0 and 1
        )

    def reparameterise(self, mean_z, log_var_z):
        if self.training:
            # eps = normal(loc=0, scale=1, size=(len(graphs.nodes), self.latent_dim=2d))
            # since variances only positive, computing log allows you to output full real range for encoder
            eps = normal(0, 1, size=(len(input_nodes), latent_dims=2d))
            z = mean_z + eps * sqrt(exp(log_var_z))
            return z
        else:
            return mean_z

    def forward(self, x):
        # reshape input into a vector, then reshape using view(-1, batchsize=2, d)
        params_z = self.encoder(x.view(-1, input_size)).view(-1, 2, d)
        
        mean_z = params_z[:, 0, :]
        log_var_z = params_z[:, 1, :]
        z = self.reparameterise(mean_z, log_var_z)
        return self.decoder(z), mean_z, log_var_z 

model = VAE().to(device)

In [ ]:
# setting optimiser
learning_rate = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

# reconstruction + KL divergence losses summed over all elements
def loss_function(x_hat, x, mean_z, log_var_z, beta):
    # binary cross entropy between input and reconstruction
    BCE = nn.functional.binary_cross_entropy(x_hat, x.view(-1, 784), reduction='sum') 
    # kl divergence: var is linear, - log var is logarithmic, mean is squared 
    KLD = 0.5 * torch.sum(exp(log_var_z) - log_var_z - 1 + mean_z**2)
    return BCE + beta * KLD

In [ ]:
# training and testing the VAE
epochs = 5
codes = dict(mean=list(), log_var=list(), y=list())
for epoch in range(0, epochs+1):
    # training
    if epoch > 0:
        model.train()
        train_loss = 0
        for x, _ in train_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            loss = loss_function(x_hat, x, mean, log_var)
            train_loss += loss.item()
            # === backward ===
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        # === log ===
        print(f'====> Epoch: {epoch} Average loss: {
            train_loss / len(train_loader.dataset):.4f}')
        
    # testing
    means, log_vars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, y in test_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            test_loss += loss_function(x_hat, x, mean, log_var).item()
            # === log ===
            means.append(mean.detach())
            log_vars.append(log_var.detach())
            labels.append(y.detach())
    # === log ===
    codes['mean'].append(torch.cat(means))
    codes['log_var'].append(torch.cat(log_vars))
    codes['y'].append(torch.cat(labels))
    test_loss /= len(test_loader.dataset)
    print(f'===> Test set loss: {test_loss:.4f}')
    display_images(x, x_hat, 1, f'Epoch {epoch}')


In [ ]:
# generating a few samples
N = 16
z = torch.randn((N, d)).to(device)
sample = model.decoder(z)
display_images(None, sample, N//4, count=True)

# Choose starting and ending point for the interpolation -> shows original and reconstructed

A, B = 1, 14
sample = model.decoder(torch.stack((mean[A].data, mean[B].data), 0))
display_images(None, torch.stack(((
    x[A].data.view(-1),
    x[B].data.view(-1),
    sample.data[0],
    sample.data[1]
)), 0))